In [71]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [72]:
cam = 2
fps = 240

model_version = "my_model_18.0.keras"
Im_Width = 128
Im_Height = 128

In [73]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [74]:
def measure_brightness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [75]:
def measure_edge_richness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    edges = cv2.Canny(gray, 200, 400)  # Apply Canny edge detection
    edge_richness = np.sum(edges)  # Sum of edge pixel intensities
    return edge_richness

In [76]:
def select_best_frame(frame_buffer):

    best_frame = None
    highest_brightness = 0
    #highest_edge_richness = 100000

    for frame in frame_buffer:

        #Crop
        cropped_frame = crop_image(frame, 220, 0, 200, 200)

        # Measure brightness
        brightness = measure_brightness(cropped_frame)

        if brightness > 150:
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame
        
        elif brightness > 25 and brightness < 90:
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame


    return best_frame  # Returns None if no frames meet the threshold


In [77]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [78]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

   # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        #send_to_arduino(arduino, 1)
        return "Non-Defective", prediction       
        
    else:
        #send_to_arduino(arduino, 0)
        return "Defective", prediction
    


In [79]:
def capture(cam, model_path, target_size=(Im_Width, Im_Height)):

    cap = cv2.VideoCapture(cam)  # Open webcam
    frame_buffer = []
    max_frames = 20  # Number of frames to evaluate

    # Load the model
    print("Loading model...") 
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:

        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        frame_buffer.append(frame)

        # Process the buffer when it's full
        if len(frame_buffer) == max_frames:
            best_frame = select_best_frame(frame_buffer)
            frame_buffer = []  # Clear the buffer

            # Process and predict
            if best_frame is not None:
                feedback, confidence = predict_frame(best_frame, model, target_size)
                print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

                prob = abs(round(confidence,2)- 0.5)*200

                # Display feedback on the frame
                cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(best_frame, f"Confidence: {prob:.0f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Display the best frame
                cv2.imshow("Best Frame", best_frame)

            else:
                # Load and display the "no-bottle" image in the same window
                image_path = "Image0001.jpg"
                image = cv2.imread(image_path)

                if image is None:
                    print("Error: Could not load 'no-bottle' image.")
                else:
                    cv2.putText(image, "No Bottle!", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.imshow("Best Frame", image)

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


In [80]:
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    capture(cam, model_path, target_size=(Im_Width, Im_Height))

Loading model...
Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
Raw Prediction: 0.5909385681152344
Prediction: Non-Defective (Confidence: 0.59)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Raw Prediction: 0.6638904809951782
Prediction: Non-Defective (Confidence: 0.66)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Raw Prediction: 0.5518721342086792
Prediction: Non-Defective (Confidence: 0.55)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Raw Prediction: 0.6157424449920654
Prediction: Non-Defective (Confidence: 0.62)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Raw Prediction: 0.6226192712783813
Prediction: Non-Defective (Confidence: 0.62)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Raw Prediction: 1.913395317387767e-05
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Raw Prediction: 1.1321758393023629e-05
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Raw Prediction: 0.0002248434175271541
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━